In [2]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")


# Pytorch device specific configuration
# Pytorch Gpu Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
# import dataset
import pandas as pd

iris = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
)

# convert from pandas dataframe to tensor
data = torch.tensor(iris[iris.columns[0:4]].values).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa'] = 0 # don't need!
labels[iris.species == "versicolor"] = 1
labels[iris.species == "virginica"] = 2

In [5]:
# Batch Size
batch_size = 16
# Use scikit learn train_test_split to split the data
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)

# Convert data into pytorch dataset
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Translate train and test data to dataloder objects
train_loader = DataLoader(train_data,batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_loader = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0], generator=torch.Generator(device=device))

In [6]:
# Create the model class
class ANNModelClass(nn.Module):
    def __init__(self, dr):
        super().__init__()

        # Layers
        self.input = nn.Linear(4, 128)
        self.hidden1 = nn.Linear(128,256)
        self.hidden2 = nn.Linear(256,128)
        self.output = nn.Linear(128, 3)

        # Parameters
        self.dr = dr
    #forward pass
    def forward(self, x):

        # input Layer
        x = F.relu(self.input(x))
        x = F.dropout(x, p=self.dr, training=self.training) # self.training turns off dropout in testing mode

        # Hidden Layer 1
        x = F.relu(self.hidden1(x))
        x = F.dropout(x, p=self.dr, training=self.training)

        # Hidden Layer 2
        x = F.relu(self.hidden2(x))
        x = F.dropout(x, p=self.dr, training=self.training)

        # Output Layer
        x = self.output(x)

        return x